~ 2014_06
~ 2019_09


## 2019_12 이후

In [27]:
import re
import pandas as pd
from pathlib import Path

# ===============================
# 1. 기본 설정
# ===============================

DATA_DIR = Path("data/")
FILENAME_PATTERN = r"REGI_FOR_(\d{4})_(\d{2})\.xlsx"

BASE_YEAR, BASE_MONTH = 2019, 12

OUTPUT_TOTAL = "E9_panel_total_allmonths.csv"
OUTPUT_BY_GENDER = "E9_panel_by_gender_allmonths.csv"


# ===============================
# 2. 보조 함수 정의
# ===============================

def parse_year_month_from_filename(path: Path):
    """파일 이름에서 연도와 월을 추출"""
    m = re.search(FILENAME_PATTERN, path.name)
    if not m:
        return None, None
    return int(m.group(1)), int(m.group(2))


def find_header_row(df_raw: pd.DataFrame):
    """'E9' 또는 '비전문취업'이 들어간 행을 찾아 그 인덱스를 반환"""
    mask = df_raw.apply(lambda x: x.astype(str).str.contains("E9|비전문취업", na=False), axis=1)
    idx = df_raw[mask.any(axis=1)].index
    if len(idx) == 0:
        raise ValueError("E9 또는 비전문취업이 포함된 헤더 행을 찾을 수 없습니다.")
    return idx[0]


def read_with_e9_header(path: Path):
    """첫 번째 시트를 읽고, 'E9'이 포함된 행을 헤더로 설정하여 DataFrame 반환"""
    temp = pd.read_excel(path, sheet_name=0, header=None)
    header_row = find_header_row(temp)
    df = pd.read_excel(path, sheet_name=0, header=[header_row])

    # 다중 헤더 처리
    if isinstance(df.columns, pd.MultiIndex):
        new_cols = []
        for top, bottom in df.columns:
            if pd.notna(bottom) and "Unnamed" not in str(bottom):
                new_cols.append(str(bottom))
            else:
                new_cols.append(str(top))
        df.columns = new_cols
    else:
        df.columns = [str(c) for c in df.columns]

    return df

def extract_e9_panel(df: pd.DataFrame, year: int, month: int):
    """시도, 시군구, 성별, E9 열만 추출 → 패널 구조로 변환"""
    def has(s, kw):
        return kw in s if isinstance(s, str) else False

    # 핵심 컬럼 찾기
    region_cols = [c for c in df.columns if has(c, "시도") or has(c, "시군구")]
    gender_col = [c for c in df.columns if has(c, "성별")]
    e9_col = [c for c in df.columns if has(c, "E9") or has(c, "비전문취업")]

    if len(region_cols) < 2 or len(gender_col) != 1 or len(e9_col) != 1:
        raise ValueError(f"열 식별 실패: region={region_cols}, gender={gender_col}, e9={e9_col}")

    # 서브셋/표준화
    df_e9 = df[region_cols + gender_col + e9_col].copy()
    df_e9.columns = ["시도", "시군구", "성별", "E9_값"]
    df_e9 = df_e9.dropna(subset=["시도", "시군구"])

    # '총계' 포함된 시도/시군구 제거
    df_e9 = df_e9[~df_e9["시도"].astype(str).str.contains("총계")]
    df_e9 = df_e9[~df_e9["시군구"].astype(str).str.contains("총계")]

    # 시군구명이 '시'/'군'/'구'로 끝나는 행만 유지
    df_e9 = df_e9[df_e9["시군구"].astype(str).str.endswith(("시", "군", "구"))]

    # 수치화
    df_e9["E9_값"] = pd.to_numeric(df_e9["E9_값"], errors="coerce").fillna(0).astype(int)

    # 성별 패널(그대로 저장)
    panel_by_gender = df_e9.copy()
    panel_by_gender["연도"] = year
    panel_by_gender["월"] = month

    # 총계 생성: 1) 시군구 단위 '성별=총계'가 있으면 사용, 2) 없으면 남성+여성 합으로 계산
    has_sgg_total = (df_e9["성별"] == "총계").any()
    if has_sgg_total:
        panel_total = (
            df_e9[df_e9["성별"] == "총계"][["시도", "시군구", "E9_값"]]
            .rename(columns={"E9_값": "E9_총계"})
            .copy()
        )
    else:
        panel_total = (
            df_e9[df_e9["성별"].isin(["남성", "여성"])]
            .groupby(["시도", "시군구"], as_index=False)["E9_값"]
            .sum()
            .rename(columns={"E9_값": "E9_총계"})
        )

    panel_total["연도"] = year
    panel_total["월"] = month

    return panel_total, panel_by_gender

# ===============================
# 3. 메인 루프 (여러 파일 반복 처리)
# ===============================

all_total = []
all_gender = []
failed = []

for file_path in sorted(DATA_DIR.glob("REGI_FOR_*.xlsx")):
    year, month = parse_year_month_from_filename(file_path)
    if year is None:
        continue

    # 2019-12 이전 파일은 건너뜀
    if (year < BASE_YEAR) or (year == BASE_YEAR and month < BASE_MONTH):
        continue

    try:
        df = read_with_e9_header(file_path)
        total_df, gender_df = extract_e9_panel(df, year, month)
        all_total.append(total_df)
        all_gender.append(gender_df)
        print(f"처리 완료: {file_path.name} (연도={year}, 월={month}, 행={len(total_df)})")

    except Exception as e:
        failed.append((file_path.name, str(e)))
        print(f"오류: {file_path.name} → {e}")


# ===============================
# 4. 결과 저장
# ===============================

if all_total:
    out_total = pd.concat(all_total, ignore_index=True)
    out_total.to_csv(OUTPUT_TOTAL, index=False, encoding="utf-8-sig")
    print(f"\n총계 패널 저장 완료 → {OUTPUT_TOTAL} (행 수: {len(out_total)})")

if all_gender:
    out_gender = pd.concat(all_gender, ignore_index=True)
    out_gender.to_csv(OUTPUT_BY_GENDER, index=False, encoding="utf-8-sig")
    print(f"성별 패널 저장 완료 → {OUTPUT_BY_GENDER} (행 수: {len(out_gender)})")

if failed:
    print("\n처리 실패 파일 목록:")
    for name, msg in failed:
        print(f"- {name}: {msg}")


처리 완료: REGI_FOR_2019_12.xlsx (연도=2019, 월=12, 행=252)
처리 완료: REGI_FOR_2020_03.xlsx (연도=2020, 월=3, 행=252)
처리 완료: REGI_FOR_2020_06.xlsx (연도=2020, 월=6, 행=251)
처리 완료: REGI_FOR_2020_09.xlsx (연도=2020, 월=9, 행=250)
처리 완료: REGI_FOR_2020_12.xlsx (연도=2020, 월=12, 행=250)
처리 완료: REGI_FOR_2021_03.xlsx (연도=2021, 월=3, 행=249)
처리 완료: REGI_FOR_2021_06.xlsx (연도=2021, 월=6, 행=249)
처리 완료: REGI_FOR_2021_09.xlsx (연도=2021, 월=9, 행=249)
처리 완료: REGI_FOR_2021_12.xlsx (연도=2021, 월=12, 행=250)
처리 완료: REGI_FOR_2022_03.xlsx (연도=2022, 월=3, 행=249)
처리 완료: REGI_FOR_2022_06.xlsx (연도=2022, 월=6, 행=249)
처리 완료: REGI_FOR_2022_09.xlsx (연도=2022, 월=9, 행=249)
처리 완료: REGI_FOR_2022_12.xlsx (연도=2022, 월=12, 행=249)
처리 완료: REGI_FOR_2023_03.xlsx (연도=2023, 월=3, 행=249)
처리 완료: REGI_FOR_2023_06.xlsx (연도=2023, 월=6, 행=249)
처리 완료: REGI_FOR_2023_09.xlsx (연도=2023, 월=9, 행=249)
처리 완료: REGI_FOR_2023_12.xlsx (연도=2023, 월=12, 행=249)
처리 완료: REGI_FOR_2024_03.xlsx (연도=2024, 월=3, 행=255)
처리 완료: REGI_FOR_2024_06.xlsx (연도=2024, 월=6, 행=254)
처리 완료: REGI_FOR_2024_09.xl

## 

In [10]:
import re
import pandas as pd
from pathlib import Path

DATA_DIR = Path("data/")
FILENAME_PATTERN = r"REGI_FOR_(\d{4})_(\d{2})\.(xls|xlsx)"
START_Y, START_M = 2014, 9
END_Y, END_M = 2019, 9
OUT_TOTAL = "E9_panel_total_2014_09_2019_09.csv"
OUT_GENDER = "E9_panel_by_gender_2014_09_2019_09.csv"

# ---------------------------
# 1. 유틸
# ---------------------------
def ym_from_name(path):
    m = re.search(FILENAME_PATTERN, path.name)
    if not m:
        return None, None
    return int(m.group(1)), int(m.group(2))

def in_range(y, m, y0, m0, y1, m1):
    return (y > y0 or (y == y0 and m >= m0)) and (y < y1 or (y == y1 and m <= m1))

def flatten_columns(cols):
    """다층 헤더일 때 읽기 쉽게 평탄화"""
    if isinstance(cols, pd.MultiIndex):
        new_cols = []
        for top, bottom in cols:
            if pd.notna(bottom) and "Unnamed" not in str(bottom):
                new_cols.append(str(bottom))
            else:
                new_cols.append(str(top))
        return new_cols
    return [str(c) for c in cols]

def find_header_row(df, max_scan=15):
    """상단 15행 내에서 '시도, 시군구, 성별, E9' 모두 포함된 행을 헤더로"""
    for r in range(min(max_scan, len(df))):
        row = "|".join(df.iloc[r].astype(str))
        if all(k in row for k in ["시", "성별"]) and ("E-9" in row or "비전문" in row):
            return r
    return None

# ---------------------------
# 2. 파서
# ---------------------------
def parse_old_format_any(path: Path, year: int, month: int):
    xls = pd.ExcelFile(path)
    for sheet in xls.sheet_names:
        raw = pd.read_excel(path, sheet_name=sheet, header=None)
        hdr_row = find_header_row(raw)
        if hdr_row is None:
            continue

        df = pd.read_excel(path, sheet_name=sheet, header=hdr_row)
        df.columns = flatten_columns(df.columns)

        # 필요한 열 찾기
        col_si  = next((c for c in df.columns if "시도" in c or "광역시도" in c), None)
        col_sgg = next((c for c in df.columns if "시군구" in c or "시군" in c), None)
        col_sex = next((c for c in df.columns if "성별" in c), None)
        col_e9  = next((c for c in df.columns if "E-9" in c or "비전문" in c), None)

        if not all([col_si, col_sgg, col_sex, col_e9]):
            continue

        sub = df[[col_si, col_sgg, col_sex, col_e9]].copy()
        sub.columns = ["시도", "시군구", "성별", "E9_값"]

        sub["시도"] = sub["시도"].ffill()

        # 총계/합계 제거
        for col in ["시도", "시군구"]:
            sub = sub[~sub[col].astype(str).str.contains("총계|소계|합계", na=False)]

        # 시군구명이 '시, 군, 구'로 끝나는 행만
        sub = sub[sub["시군구"].astype(str).str.endswith(("시", "군", "구"))]
        sub["E9_값"] = pd.to_numeric(sub["E9_값"], errors="coerce").fillna(0).astype(int)

        panel_by_gender = sub.copy()
        panel_by_gender["연도"] = year
        panel_by_gender["월"] = month

        is_total = panel_by_gender["성별"].astype(str).str.contains("계|총계")
        panel_total = (panel_by_gender.loc[is_total, ["시도", "시군구", "E9_값"]]
                       .rename(columns={"E9_값": "E9_총계"}))
        panel_total["연도"] = year
        panel_total["월"] = month

        return panel_total, panel_by_gender

    raise ValueError("적절한 헤더를 찾지 못했습니다.")

# ---------------------------
# 3. 메인 루프
# ---------------------------
all_total, all_gender, failed = [], [], []
files = sorted(list(DATA_DIR.glob("REGI_FOR_*.xls")) + list(DATA_DIR.glob("REGI_FOR_*.xlsx")))

for path in files:
    y, m = ym_from_name(path)
    if y is None or not in_range(y, m, START_Y, START_M, END_Y, END_M):
        continue
    try:
        pt, pg = parse_old_format_any(path, y, m)
        all_total.append(pt); all_gender.append(pg)
        print(f"처리 완료: {path.name} (연도={y}, 월={m}, 행={len(pt)})")
    except Exception as e:
        failed.append((path.name, str(e)))
        print(f"오류: {path.name} → {e}")

# ---------------------------
# 4. 저장
# ---------------------------
if all_total:
    out_total = pd.concat(all_total, ignore_index=True)
    out_total.to_csv(OUTPUT_TOTAL, index=False, encoding="utf-8-sig")
    print(f"\n총계 패널 저장 완료 → {OUTPUT_TOTAL} (행 수: {len(out_total)})")

if all_gender:
    out_gender = pd.concat(all_gender, ignore_index=True)
    out_gender.to_csv(OUTPUT_BY_GENDER, index=False, encoding="utf-8-sig")
    print(f"성별 패널 저장 완료 → {OUTPUT_BY_GENDER} (행 수: {len(out_gender)})")

if failed:
    print("\n처리 실패 파일 목록:")
    for name, msg in failed:
        print(f"- {name}: {msg}")


처리 완료: REGI_FOR_2014_09.xlsx (연도=2014, 월=9, 행=258)
처리 완료: REGI_FOR_2014_12.xlsx (연도=2014, 월=12, 행=258)
처리 완료: REGI_FOR_2015_03.xlsx (연도=2015, 월=3, 행=257)
처리 완료: REGI_FOR_2015_06.xls (연도=2015, 월=6, 행=253)
처리 완료: REGI_FOR_2015_09.xls (연도=2015, 월=9, 행=253)
처리 완료: REGI_FOR_2015_12.xls (연도=2015, 월=12, 행=258)
처리 완료: REGI_FOR_2016_03.xls (연도=2016, 월=3, 행=253)
처리 완료: REGI_FOR_2016_06.xls (연도=2016, 월=6, 행=253)
처리 완료: REGI_FOR_2016_09.xls (연도=2016, 월=9, 행=251)
처리 완료: REGI_FOR_2016_12.xls (연도=2016, 월=12, 행=251)
처리 완료: REGI_FOR_2017_03.xls (연도=2017, 월=3, 행=251)
처리 완료: REGI_FOR_2017_06.xls (연도=2017, 월=6, 행=250)
처리 완료: REGI_FOR_2017_09.xls (연도=2017, 월=9, 행=250)
처리 완료: REGI_FOR_2017_12.xls (연도=2017, 월=12, 행=250)
처리 완료: REGI_FOR_2018_03.xls (연도=2018, 월=3, 행=250)
처리 완료: REGI_FOR_2018_06.xls (연도=2018, 월=6, 행=250)
처리 완료: REGI_FOR_2018_09.xlsx (연도=2018, 월=9, 행=251)
처리 완료: REGI_FOR_2018_12.xlsx (연도=2018, 월=12, 행=251)
처리 완료: REGI_FOR_2019_03.xlsx (연도=2019, 월=3, 행=251)
처리 완료: REGI_FOR_2019_06.xlsx (연도=2019, 

##

In [13]:
import re
import pandas as pd
from pathlib import Path

# ==============================
# 설정
# ==============================
DATA_DIR = Path("data/")  # 파일 폴더
FILENAME_PATTERN = r"REGI_FOR_(\d{4})_(\d{2})\.(xls|xlsx)"

# 처리 범위: 초기형 (2012-12 ~ 2014-06)
START_Y, START_M = 2012, 12
END_Y, END_M = 2014, 6

OUT_TOTAL  = "E9_panel_total_2012_12_2014_06.csv"
OUT_GENDER = "E9_panel_by_gender_2012_12_2014_06.csv"

# ==============================
# 유틸
# ==============================
TOTAL_WORDS = re.compile(r"(총계|총합계|소계|합계)")

def ym_from_name(path: Path):
    m = re.search(FILENAME_PATTERN, path.name)
    return (int(m.group(1)), int(m.group(2))) if m else (None, None)

def in_range(y, m, y0, m0, y1, m1):
    return (y > y0 or (y == y0 and m >= m0)) and (y < y1 or (y == y1 and m <= m1))

def _to_int_safe(x):
    if pd.isna(x):
        return 0
    s = str(x).strip().replace(",", "")
    try:
        return int(float(s))
    except Exception:
        return 0

def _split3_to_tuple(cell_text):
    """
    '계\\n남\\n여' 형태의 셀을 (계, 남, 여) 튜플로 변환.
    불완전/결측은 0으로 보정.
    """
    if pd.isna(cell_text):
        return (0, 0, 0)
    parts = [p.strip() for p in str(cell_text).split("\n")]
    vals = [ _to_int_safe(p) for p in parts ] + [0,0,0]
    return (vals[0], vals[1], vals[2])

def _find_header_row(df, max_scan=30):
    """
    상단에서 '시도/광역시도', '시군구', '성별'과 'E-9/비전문' 키워드가
    함께 포함된 줄을 헤더로 선택.
    """
    for r in range(min(max_scan, len(df))):
        row = "|".join(df.iloc[r].astype(str))
        if (("시도" in row) or ("광역시도" in row)) and ("시군구" in row) and ("성별" in row) \
           and (("비전문" in row) or ("E-9" in row) or ("E9" in row)):
            return r
    # 못 찾으면 최상단(0행)으로 fallback (2012-12 유형)
    return 0

# ==============================
# 초기형 파서: E-9 관련 모든 열 합산
# ==============================
def parse_early_format_sum_all_e9(path: Path, year: int, month: int):
    """
    초기형(2012-12 ~ 2014-06) 전용 파서:
      - 첫 시트에서 헤더 자동탐색
      - 'E-9' 문자열(하이픈/스페이스 변형 포함) 및 '비전문'이 들어간 모든 열을 수집
      - 각 E-9 열의 '계\\n남\\n여' 값을 튜플로 파싱 후, 행 단위로 계/남/여 합산
      - 시도 ffill, 합계성(총계/소계/합계 등) 제거, '시/군/구'로 끝나는 행만 유지
      - panel_total(성별=계), panel_by_gender(계/남/여 롱) 반환
    """
    # 1) 헤더 위치 탐색 후 읽기
    raw = pd.read_excel(path, sheet_name=0, header=None, dtype=str)
    hdr = _find_header_row(raw)
    df  = pd.read_excel(path, sheet_name=0, header=hdr, dtype=str)

    # 2) 기본 열 식별
    cols = [str(c) for c in df.columns]
    col_si  = next((c for c in cols if ("시도" in c) or ("광역시도" in c)), None)
    col_sgg = next((c for c in cols if "시군구" in c), None)
    if not all([col_si, col_sgg]):
        raise ValueError(f"필요 열 없음: si={col_si}, sgg={col_sgg}")

    # 3) E-9 관련 모든 열 찾기 (총계 + 세부분류)
    #    하이픈/대시/스페이스 변형 허용: E9, E-9, E – 9 등
    rx_e9 = re.compile(r"E\s*[-–—]?\s*9", re.I)
    e9_cols = [c for c in cols if ("비전문" in c) or rx_e9.search(c)]
    if not e9_cols:
        raise ValueError("E-9 관련 열을 찾지 못했습니다.")

    sub = df[[col_si, col_sgg] + e9_cols].copy()
    sub.columns = ["시도", "시군구"] + e9_cols

    # 4) 시도 보정, 합계성/비정규 행 제거
    sub["시도"] = sub["시도"].ffill()
    for c in ["시도", "시군구"]:
        sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
    sub = sub[sub["시군구"].astype(str).str.endswith(("시", "군", "구"))]

    # 5) 모든 E-9 열 → (계, 남, 여) 튜플로 변환 후, 행 단위 합산
    triples = {c: sub[c].apply(_split3_to_tuple) for c in e9_cols}
    total_계 = sum(triples[c].apply(lambda t: t[0]) for c in e9_cols)
    total_남 = sum(triples[c].apply(lambda t: t[1]) for c in e9_cols)
    total_여 = sum(triples[c].apply(lambda t: t[2]) for c in e9_cols)

    sub_out = sub[["시도", "시군구"]].copy()
    sub_out["계"] = total_계.astype(int)
    sub_out["남"] = total_남.astype(int)
    sub_out["여"] = total_여.astype(int)

    # 6) 성별 롱 포맷
    panel_by_gender = (
        sub_out.melt(id_vars=["시도", "시군구"], value_vars=["계", "남", "여"],
                     var_name="성별", value_name="E9_값")
        .assign(연도=year, 월=month)
        .reset_index(drop=True)
    )

    # 7) 총계 패널(성별=계)
    panel_total = (
        panel_by_gender.query("성별 == '계'")[["시도", "시군구", "E9_값", "연도", "월"]]
        .rename(columns={"E9_값": "E9_총계"})
        .reset_index(drop=True)
    )

    return panel_total, panel_by_gender

# ==============================
# 실행 루프 (초기형만)
# ==============================
def main():
    all_total, all_gender, failed = [], [], []

    files = sorted(list(DATA_DIR.glob("REGI_FOR_*.xls")) + list(DATA_DIR.glob("REGI_FOR_*.xlsx")))
    for path in files:
        y, m = ym_from_name(path)
        if y is None or not in_range(y, m, START_Y, START_M, END_Y, END_M):
            continue
        try:
            pt, pg = parse_early_format_sum_all_e9(path, y, m)
            all_total.append(pt)
            all_gender.append(pg)
            print(f"처리 완료: {path.name} (연도={y}, 월={m}, 총계행={len(pt)}, 성별행={len(pg)})")
        except Exception as e:
            failed.append((path.name, str(e)))
            print(f"오류: {path.name} → {e}")

    if all_total:
        pd.concat(all_total, ignore_index=True).to_csv(OUT_TOTAL, index=False, encoding="utf-8-sig")
        print(f"\n총계 패널 저장: {OUT_TOTAL} (행 수: {sum(len(df) for df in all_total)})")
    if all_gender:
        pd.concat(all_gender, ignore_index=True).to_csv(OUT_GENDER, index=False, encoding="utf-8-sig")
        print(f"성별 패널 저장: {OUT_GENDER} (행 수: {sum(len(df) for df in all_gender)})")

    if failed:
        print("\n처리 실패 파일:")
        for n, msg in failed:
            print(f"- {n}: {msg}")

if __name__ == "__main__":
    main()


/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]


처리 완료: REGI_FOR_2012_12.xlsx (연도=2012, 월=12, 총계행=258, 성별행=774)


/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expr

처리 완료: REGI_FOR_2013_03.xlsx (연도=2013, 월=3, 총계행=259, 성별행=777)
처리 완료: REGI_FOR_2013_06.xlsx (연도=2013, 월=6, 총계행=259, 성별행=777)


/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expr

처리 완료: REGI_FOR_2013_09.xlsx (연도=2013, 월=9, 총계행=259, 성별행=777)
처리 완료: REGI_FOR_2013_12.xlsx (연도=2013, 월=12, 총계행=259, 성별행=777)
처리 완료: REGI_FOR_2014_03.xlsx (연도=2014, 월=3, 총계행=257, 성별행=771)
처리 완료: REGI_FOR_2014_06.xlsx (연도=2014, 월=6, 총계행=257, 성별행=771)

총계 패널 저장: E9_panel_total_2012_12_2014_06.csv (행 수: 1808)
성별 패널 저장: E9_panel_by_gender_2012_12_2014_06.csv (행 수: 5424)


/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sub = sub[~sub[c].astype(str).str.contains(TOTAL_WORDS, na=False)]
/var/folders/fq/fzkykm4d22s80m_f4knndwb80000gn/T/ipykernel_24668/1977083554.py:100: UserWarning: This pattern is interpreted as a regular expr

In [24]:
import re
import unicodedata
import pandas as pd
from pathlib import Path

# ==============================
# 설정
# ==============================
DATA_DIR = Path("data/")  # 파일 폴더
FILENAME_PATTERN = r"REGI_FOR_(\d{4})_(\d{2})\.(xls|xlsx)"

# 처리 범위: 2009-01 ~ 2012-12
START_Y, START_M = 2009, 6
END_Y, END_M = 2012, 6

OUT_TOTAL  = "E9_panel_total_2009_06_2012_06.csv"
OUT_GENDER = "E9_panel_by_gender_2009_06_2012_06.csv"

# ==============================
# 패턴/토큰
# ==============================
TOTAL_WORDS = re.compile(r"(총계|총합계|소계|합계)")
MALE_TOKENS   = {"M", "m", "남", "남자", "남성", "Male"}
FEMALE_TOKENS = {"F", "f", "여", "여자", "여성", "Female"}

# 자격명/코드 신호 (헤더 자동 탐색에 사용)
VISA_KR_TOKENS = [
    "비전문취업","전문인력","방문취업","유학","어학연수","결혼이민","재외동포",
    "단기취업","주재","기업투자","교환학생","연구","교수","취업","예능","흥행","연수"
]
VISA_CODE_RX = re.compile(r"\b[DEHFQRTMZ]\s*[-–—]?\s*\d{1,2}\b", re.I)
RX_E9 = re.compile(r"E\s*[-–—]?\s*9", re.I)   # E9, E-9, E – 9 허용

# ==============================
# 유틸
# ==============================
def ym_from_name(path: Path):
    m = re.search(FILENAME_PATTERN, path.name)
    return (int(m.group(1)), int(m.group(2))) if m else (None, None)

def in_range(y, m, y0, m0, y1, m1):
    return (y > y0 or (y == y0 and m >= m0)) and (y < y1 or (y == y1 and m <= m1))

def _norm(s: str) -> str:
    t = unicodedata.normalize("NFKC", str(s))
    for ch in ["\u200b", "\xa0", "\u3000"]:
        t = t.replace(ch, "")
    return t.strip()

def to_int(s):
    if pd.isna(s): return 0
    s = str(s).replace(",", "").strip()
    if s == "": return 0
    try: return int(float(s))
    except: return 0

def is_province(name: str) -> bool:
    if not isinstance(name, str): return False
    s = name.strip()
    if any(k in s for k in ["광역시", "특별시", "특별자치시", "특별자치도"]): return True
    return s.endswith("도")  # 경기도/강원도 등

def is_sigungu(name: str) -> bool:
    if not isinstance(name, str): return False
    s = name.strip()
    return s.endswith(("시","군","구"))
    # 읍/면/동까지 포함하려면: return s.endswith(("시","군","구","읍","면","동"))

# ==============================
# 헤더: 자격명(체류자격) 기반 자동 탐색
# ==============================
def _row_has_visa_signals(values) -> int:
    score = 0
    for v in values:
        s = _norm(v)
        if not s or s == "구분":
            continue
        if VISA_CODE_RX.search(s):
            score += 2
        elif any(tok in s for tok in VISA_KR_TOKENS):
            score += 1
    return score

def read_with_visa_header(path: Path, sheet=0, scan_rows=14, min_score=3):
    """
    상단 scan_rows 행을 훑어 '자격명/코드(E-9 등)' 신호가 많은 행을 헤더로 선택.
    '구분' 존재 여부는 전혀 사용하지 않음.
    """
    raw = pd.read_excel(path, sheet_name=sheet, header=None, dtype=str)
    best_r, best_score = None, -1
    limit = min(scan_rows, len(raw))
    for r in range(limit):
        vals = raw.iloc[r].tolist()
        sc = _row_has_visa_signals(vals)
        if sc > best_score:
            best_r, best_score = r, sc

    candidates = []
    if best_r is not None and best_score >= min_score:
        candidates.append(best_r)
    # 흔히 쓰이는 위치들도 후보로 추가
    candidates.extend([3, 2, 4, 1, 5, 0])
    seen = set()
    candidates = [h for h in candidates if (h not in seen and not seen.add(h) and 0 <= h < len(raw))]

    last_err = None
    for hdr in candidates:
        try:
            df = pd.read_excel(path, sheet_name=sheet, header=hdr, dtype=str)
            cols = [str(c) for c in df.columns]
            visa_like = [c for c in cols if VISA_CODE_RX.search(c) or any(tok in c for tok in VISA_KR_TOKENS)]
            if visa_like:
                return df, hdr
        except Exception as e:
            last_err = e
    raise ValueError(f"자격명 기반 헤더 식별 실패 (best_row={best_r}, score={best_score})") from last_err

# ==============================
# 라벨(지역/성별) 컬럼 자동 선택
# ==============================
def pick_label_column(df: pd.DataFrame) -> str:
    """
    '구분'이 없을 때, 어떤 컬럼이 지역/성별 라벨을 담고 있는지 휴리스틱으로 선택.
    - 시도/시군구/성별 토큰이 가장 많이 등장하는 컬럼을 선택
    """
    best_col, best_score = None, -1
    sample_rows = min(80, len(df))
    for c in df.columns:
        score = 0
        col = df[c].astype(str).head(sample_rows)
        for val in col:
            s = _norm(val)
            if not s:
                continue
            if is_province(s): score += 3
            if is_sigungu(s):  score += 3
            if s in MALE_TOKENS or s in FEMALE_TOKENS: score += 2
            if TOTAL_WORDS.search(s): score += 1
        if score > best_score:
            best_col, best_score = c, score
    if best_col is None:
        raise ValueError("라벨(지역/성별) 컬럼을 식별하지 못했습니다.")
    return best_col

# ==============================
# E-9 열 수집
# ==============================
def collect_e9_columns(df: pd.DataFrame):
    return [c for c in df.columns if ("비전문" in str(c)) or RX_E9.search(str(c))]

# ==============================
# 초기형 포맷(셀에 '계\\n남\\n여') 감지 및 파싱
# ==============================
def is_multiline_gender_format(df: pd.DataFrame, e9_cols: list, probe=50) -> bool:
    """
    E-9 열 중 일부 셀에 '\n' 이 포함되면 초기형(계/남/여 한 셀) 포맷으로 판단
    """
    n = min(probe, len(df))
    for c in e9_cols:
        s = df[c].astype(str).head(n)
        if any(("\n" in x) for x in s if pd.notna(x)):
            return True
    return False

def split3_tuple(cell_text):
    if pd.isna(cell_text): return (0,0,0)
    parts = [p.strip() for p in str(cell_text).split("\n")]
    vals = [ to_int(p) for p in parts ] + [0,0,0]
    return (vals[0], vals[1], vals[2])

# ==============================
# 파서 본체: 모든 E-9 합산 + 성별/총계 출력
# ==============================
def parse_2009_2012_all(path: Path, year: int, month: int):
    # 1) 자격명 기반 헤더로 읽기 (전 파일 공통)
    df, used_hdr = read_with_visa_header(path)
    df.columns = [str(c) for c in df.columns]

    # 2) 라벨 컬럼(지역/성별) 자동 선택
    label_col = pick_label_column(df)

    # 3) E-9 관련 모든 열
    e9_cols = collect_e9_columns(df)
    if not e9_cols:
        raise ValueError("E-9 관련 열을 찾지 못했습니다.")

    rows_total, rows_gender = [], []
    current_province, current_area = None, None
    last_province_seen = None   # 최근 시군구에서 확인한 시도값

    # 4) 포맷 분기
    if is_multiline_gender_format(df, e9_cols):
        # (A) 초기형: 각 E-9 셀에 '계\\n남\\n여'
        sub = df[[label_col] + e9_cols].copy()
        sub.rename(columns={label_col: "라벨"}, inplace=True)

        # 모든 E-9 열을 (계,남,여)로 분해 후 합산
        triples = {c: sub[c].apply(split3_tuple) for c in e9_cols}
        total_계 = sum(triples[c].apply(lambda t: t[0]) for c in e9_cols)
        total_남 = sum(triples[c].apply(lambda t: t[1]) for c in e9_cols)
        total_여 = sum(triples[c].apply(lambda t: t[2]) for c in e9_cols)

        sub_out = pd.DataFrame({
            "라벨": sub["라벨"],
            "계": total_계.astype(int),
            "남": total_남.astype(int),
            "여": total_여.astype(int),
        })

        # 컨텍스트 매핑: 시도/시군구/성별
        for _, r in sub_out.iterrows():
            label = str(r["라벨"]).strip() if pd.notna(r["라벨"]) else ""
            if is_province(label):
                current_province, current_area = label, None
                continue
            if (len(label) <= 1) or TOTAL_WORDS.search(label):
                continue
            if is_sigungu(label):
                current_area = label
                if current_province is not None:
                    last_province_seen = current_province
                s_total = int(r["계"])
                rows_total.append((current_province, current_area, s_total))
                rows_gender.extend([
                    (current_province, current_area, "계", int(r["계"])),
                    (current_province, current_area, "남", int(r["남"])),
                    (current_province, current_area, "여", int(r["여"])),
                ])
                continue
            # 기타 라벨은 무시

    else:
        # (B) 숫자형: 시군구 행과 성별(F/M/남/여) 행이 분리
        sub = df[[label_col] + e9_cols].copy()
        sub.rename(columns={label_col: "라벨"}, inplace=True)

        for _, row in sub.iterrows():
            label = str(row["라벨"]).strip() if pd.notna(row["라벨"]) else ""

            # 시도
            if is_province(label):
                current_province, current_area = label, None
                continue

            # 합계성/짧은 라벨 제거
            if (len(label) <= 1) or TOTAL_WORDS.search(label):
                continue

            # 시군구
            if is_sigungu(label):
                current_area = label
                if current_province is not None:
                    last_province_seen = current_province
                s = sum(to_int(row[c]) for c in e9_cols)
                rows_total.append((current_province, current_area, s))
                rows_gender.append((current_province, current_area, "계", s))
                continue

            # 성별 행
            if label in MALE_TOKENS | FEMALE_TOKENS:
                if current_area is None:
                    continue
                s = sum(to_int(row[c]) for c in e9_cols)
                gender = "남" if label in MALE_TOKENS else "여"
                # 시도 누락 보정: 최근 본 시도값으로 대체
                province_for_gender = current_province if current_province is not None else last_province_seen
                rows_gender.append((province_for_gender, current_area, gender, s))
                continue
            # 기타 라벨은 무시

    # 5) DataFrame 구성 + 시도 보정 조인
    df_total = pd.DataFrame(rows_total, columns=["시도", "시군구", "E9_총계"]).dropna(subset=["시군구"])
    df_total["연도"] = year
    df_total["월"] = month
    df_total = df_total.reset_index(drop=True)

    df_gender = pd.DataFrame(rows_gender, columns=["시도", "시군구", "성별", "E9_값"]).dropna(subset=["시군구"])
    df_gender["연도"] = year
    df_gender["월"] = month
    df_gender = df_gender.reset_index(drop=True)

    # (추가 보정) (연도,월,시군구) 기준으로 총계 테이블에서 시도 가져와서 빈 칸 채우기
    key = ["연도", "월", "시군구"]
    df_gender = df_gender.merge(
        df_total[key + ["시도"]].drop_duplicates(),
        on=key,
        how="left",
        suffixes=("", "_from_total")
    )
    df_gender["시도"] = df_gender["시도"].fillna(df_gender["시도_from_total"])
    df_gender = df_gender.drop(columns=["시도_from_total"])

    return df_total, df_gender

# ==============================
# 실행 루프
# ==============================
def main():
    all_total, all_gender, failed = [], [], []

    files = sorted(list(DATA_DIR.glob("REGI_FOR_*.xls")) + list(DATA_DIR.glob("REGI_FOR_*.xlsx")))
    for path in files:
        y, m = ym_from_name(path)
        if y is None or not in_range(y, m, START_Y, START_M, END_Y, END_M):
            continue
        try:
            pt, pg = parse_2009_2012_all(path, y, m)
            all_total.append(pt); all_gender.append(pg)
            print(f"처리 완료: {path.name} (연도={y}, 월={m}, 총계행={len(pt)}, 성별행={len(pg)})")
        except Exception as e:
            failed.append((path.name, str(e)))
            print(f"오류: {path.name} → {e}")

    if all_total:
        out_total = pd.concat(all_total, ignore_index=True)
        out_total.to_csv(OUT_TOTAL, index=False, encoding="utf-8-sig")
        print(f"\n총계 패널 저장: {OUT_TOTAL} (행 수: {len(out_total)})")

    if all_gender:
        out_gender = pd.concat(all_gender, ignore_index=True)
        out_gender.to_csv(OUT_GENDER, index=False, encoding="utf-8-sig")
        print(f"성별 패널 저장: {OUT_GENDER} (행 수: {len(out_gender)})")

    if failed:
        print("\n처리 실패 파일:")
        for n, msg in failed:
            print(f"- {n}: {msg}")

if __name__ == "__main__":
    main()


처리 완료: REGI_FOR_2009_06.xlsx (연도=2009, 월=6, 총계행=250, 성별행=366)
처리 완료: REGI_FOR_2009_12.xlsx (연도=2009, 월=12, 총계행=252, 성별행=368)
처리 완료: REGI_FOR_2010_06.xlsx (연도=2010, 월=6, 총계행=251, 성별행=367)
처리 완료: REGI_FOR_2010_12.xls (연도=2010, 월=12, 총계행=254, 성별행=370)
처리 완료: REGI_FOR_2011_06.xls (연도=2011, 월=6, 총계행=255, 성별행=371)
처리 완료: REGI_FOR_2011_12.xls (연도=2011, 월=12, 총계행=258, 성별행=374)
처리 완료: REGI_FOR_2012_06.xls (연도=2012, 월=6, 총계행=259, 성별행=375)

총계 패널 저장: E9_panel_total_2009_06_2012_06.csv (행 수: 1779)
성별 패널 저장: E9_panel_by_gender_2009_06_2012_06.csv (행 수: 2591)


## Merge all files

In [28]:
import pandas as pd

# Load all the partial total-panel CSVs
files = [
    "E9_panel_total_2009_06_2012_06.csv",
    "E9_panel_total_2012_12_2014_06.csv",
    "E9_panel_total_2014_09_2019_09.csv",
    "E9_panel_total_allmonths.csv"
]

dfs = []
for f in files:
    try:
        df = pd.read_csv(f, dtype=str)
        print(f"Loaded {f} ({len(df)} rows)")
        dfs.append(df)
    except Exception as e:
        print(f"Failed to load {f}: {e}")

# Concatenate all into one
merged = pd.concat(dfs, ignore_index=True)

# Deduplicate if necessary
merged = merged.drop_duplicates()

# Save to a single CSV
out_path = "E9_panel_total_allmonths_merged.csv"
merged.to_csv(out_path, index=False, encoding="utf-8-sig")

out_path, merged.shape


Loaded E9_panel_total_2009_06_2012_06.csv (1779 rows)
Loaded E9_panel_total_2012_12_2014_06.csv (1808 rows)
Loaded E9_panel_total_2014_09_2019_09.csv (5301 rows)
Loaded E9_panel_total_allmonths.csv (5760 rows)


('E9_panel_total_allmonths_merged.csv', (14648, 5))